In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from pandas.tseries.offsets import MonthEnd

oil = pd.read_csv('csv_file/oil.csv')
stores = pd.read_csv('csv_file/stores.csv')
holidays_events = pd.read_csv('csv_file/holidays_events.csv')
test = pd.read_csv('csv_file/test.csv')
transactions = pd.read_csv('csv_file/transactions.csv')
train = pd.read_csv('csv_file/train.csv')
sample_submission = pd.read_csv('csv_file/sample_submission.csv')

train = train.merge(stores, how='left', on='store_nbr')
train['date'] = pd.to_datetime(train['date'])
transactions['date'] = pd.to_datetime(transactions['date'])
train = train.merge(transactions, how='left', on=['date', 'store_nbr'])
oil['date'] = pd.to_datetime(oil['date'])
train = train.merge(oil, how='left', on='date')

train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['day_of_week'] = train['date'].dt.dayofweek
train['weekend'] = train['day_of_week'].isin([5, 6]).astype(int)


test = test.merge(stores, how='left', on='store_nbr')
test['date'] = pd.to_datetime(test['date'])
test = test.merge(transactions, how='left', on=['date', 'store_nbr'])
test = test.merge(oil, how='left', on='date')


test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['day_of_week'] = test['date'].dt.dayofweek
test['weekend'] = test['day_of_week'].isin([5, 6]).astype(int)

train['transactions'].fillna(0, inplace=True)
test['transactions'].fillna(0, inplace=True)
train['family'] = train['family'].astype('category').cat.codes
test['family'] = test['family'].astype('category').cat.codes


train_with_values = train.dropna(subset=['dcoilwtico'])
train_to_fill = train[train['dcoilwtico'].isna()]
X_train = train_with_values.drop(columns=['dcoilwtico'])
y_train = train_with_values['dcoilwtico']
numeric_columns = X_train.select_dtypes(include=np.number).columns
X_train = X_train[numeric_columns]
model = LinearRegression()
model.fit(X_train, y_train)
X_to_fill = train_to_fill.drop(columns=['dcoilwtico'])[numeric_columns]
predicted_values = model.predict(X_to_fill)
train.loc[train['dcoilwtico'].isna(), 'dcoilwtico'] = predicted_values

holidays_events = holidays_events[(holidays_events['transferred'] != True) & (holidays_events['type'] != 'Work Day')]

national_holidays = holidays_events[holidays_events['locale'] == 'National']
regional_holidays = holidays_events[holidays_events['locale'] == 'Regional']
local_holidays = holidays_events[holidays_events['locale'] == 'Local']

national_holidays['date'] = pd.to_datetime(national_holidays['date'])
train = pd.merge(train, national_holidays[['date', 'locale']], on='date', how='left')
test = pd.merge(test, national_holidays[['date', 'locale']], on='date', how='left')

regional_holidays['date'] = pd.to_datetime(regional_holidays['date'])
train = pd.merge(train, regional_holidays[['date','locale_name']], left_on=['date', 'state'], right_on=['date', 'locale_name'], how='left')
test = pd.merge(test, regional_holidays[['date','locale_name']], left_on=['date', 'state'], right_on=['date', 'locale_name'], how='left')

local_holidays['date'] = pd.to_datetime(local_holidays['date'])
train = pd.merge(train, local_holidays[['date', 'locale_name']], left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left')
test = pd.merge(test, local_holidays[['date', 'locale_name']], left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left')

train['holiday'] = np.where((train['locale'].isna()) & (train['locale_name_x'].isna()) & (train['locale_name_y'].isna()), 0, 1)
test['holiday'] = np.where((test['locale'].isna()) & (test['locale_name_x'].isna()) & (test['locale_name_y'].isna()), 0, 1)

for col in ['city', 'state', 'type']:
    train[col] = train[col].astype('category').cat.codes
    test[col] = test[col].astype('category').cat.codes

train['payday'] = np.where((train['date'].dt.day == 15) | 
                            (train['date'] == pd.to_datetime(train['date'], format="%Y%m") + MonthEnd(0)), 
                            1, 0)
test['payday'] = np.where((test['date'].dt.day == 15) | 
                            (test['date'] == pd.to_datetime(test['date'], format="%Y%m") + MonthEnd(0)), 
                            1, 0)



C:\Users\agamj\AppData\Local\Temp\ipykernel_20028\695207797.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  national_holidays['date'] = pd.to_datetime(national_holidays['date'])
C:\Users\agamj\AppData\Local\Temp\ipykernel_20028\695207797.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regional_holidays['date'] = pd.to_datetime(regional_holidays['date'])
C:\Users\agamj\AppData\Local\Temp\ipykernel_20028\695207797.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [9]:
columns_to_drop = ['date', 'locale', 'locale_name_x', 'locale_name_y']

train = train.drop(columns=columns_to_drop)


test = test.drop(columns=columns_to_drop)

print("Train Shape:", train.shape)
print("Test Shape:", test.shape)
print("Train Data:")
print(train.head())
print("\nTest Data:")
print(test.head())

Train Shape: (3008280, 18)
Test Shape: (28512, 17)
Train Data:
   id  store_nbr  family  sales  onpromotion  city  state  type  cluster  \
0   0          1       0    0.0            0    18     12     3       13   
1   1          1       1    0.0            0    18     12     3       13   
2   2          1       2    0.0            0    18     12     3       13   
3   3          1       3    0.0            0    18     12     3       13   
4   4          1       4    0.0            0    18     12     3       13   

   transactions  dcoilwtico  year  month  day  day_of_week  weekend  holiday  \
0           0.0  103.199613  2013      1    1            1        0        1   
1           0.0  103.197329  2013      1    1            1        0        1   
2           0.0  103.195045  2013      1    1            1        0        1   
3           0.0  103.192761  2013      1    1            1        0        1   
4           0.0  103.190477  2013      1    1            1        0        1   


In [10]:
subset_train = train[(train['family'] == 23) & (train['store_nbr'] == 23)]

print("Subset of train where family=1 and store=1:")
print(subset_train)


Subset of train where family=1 and store=1:
              id  store_nbr  family  sales  onpromotion  city  state  type  \
518          518         23      23    0.0            0     0     15     3   
2300        2300         23      23    0.0            0     0     15     3   
4082        4082         23      23    0.0            0     0     15     3   
5864        5864         23      23    0.0            0     0     15     3   
7646        7646         23      23    0.0            0     0     15     3   
...          ...        ...     ...    ...          ...   ...    ...   ...   
2999888  2992496         23      23    8.0            0     0     15     3   
3001670  2994278         23      23   10.0            0     0     15     3   
3003452  2996060         23      23    3.0            0     0     15     3   
3005234  2997842         23      23    5.0            0     0     15     3   
3007016  2999624         23      23    4.0            0     0     15     3   

         cluster  t

In [15]:
from pathlib import Path  

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

comp_dir = Path('csv_file')

store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

family_sales = (
    store_sales
    .groupby(['family', 'date'])
    .mean()
    .unstack('family')
    .loc['2017']
)

test = pd.read_csv(
    comp_dir / 'test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
test['date'] = test.date.dt.to_period('D')
test = test.set_index(['store_nbr', 'family', 'date']).sort_index()
test_sales = (
    test
    .groupby(['family', 'date'])
    .mean()
    .unstack('family')
    .loc['2017']
)

In [16]:
def make_lags(ts, lags, lead_time=1):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)
def make_multistep_target(ts, steps):
    return pd.concat(
        {f'y_step_{i + 1}': ts.shift(-i)
         for i in range(steps)},
        axis=1)
y = family_sales.loc[:, 'sales']
print(family_sales)
X = make_lags(y, lags=4).fillna(0)

y = make_multistep_target(y, steps=16).fillna(0)

y, X = y.align(X, join='inner', axis=0)


                sales                                              \
family     AUTOMOTIVE BABY CARE     BEAUTY    BEVERAGES     BOOKS   
date                                                                
2017-01-01   0.092593  0.037037   0.055556    74.222221  0.000000   
2017-01-02  11.481482  0.259259  11.648149  6208.055664  0.481481   
2017-01-03   8.296296  0.296296   7.185185  4507.814941  0.814815   
2017-01-04   6.833333  0.333333   6.888889  3911.833252  0.759259   
2017-01-05   6.333333  0.351852   5.925926  3258.796387  0.407407   
...               ...       ...        ...          ...       ...   
2017-08-11   8.166667  0.129630   6.500000  3502.055664  0.000000   
2017-08-12   7.462963  0.055556   6.833333  3376.259277  0.000000   
2017-08-13   8.907408  0.166667   8.018518  3747.296387  0.018519   
2017-08-14   5.407407  0.166667   6.240741  3237.629639  0.000000   
2017-08-15   6.240741  0.148148   6.277778  3162.462891  0.000000   

                                 

In [17]:
print(y)
le = LabelEncoder()
X = (X
    .stack('family')  
    .reset_index('family')  
    .assign(family=lambda x: le.fit_transform(x.family))  
)
y = y.stack('family')  

             y_step_1                                              \
family     AUTOMOTIVE BABY CARE     BEAUTY    BEVERAGES     BOOKS   
date                                                                
2017-01-01   0.092593  0.037037   0.055556    74.222221  0.000000   
2017-01-02  11.481482  0.259259  11.648149  6208.055664  0.481481   
2017-01-03   8.296296  0.296296   7.185185  4507.814941  0.814815   
2017-01-04   6.833333  0.333333   6.888889  3911.833252  0.759259   
2017-01-05   6.333333  0.351852   5.925926  3258.796387  0.407407   
...               ...       ...        ...          ...       ...   
2017-08-11   8.166667  0.129630   6.500000  3502.055664  0.000000   
2017-08-12   7.462963  0.055556   6.833333  3376.259277  0.000000   
2017-08-13   8.907408  0.166667   8.018518  3747.296387  0.018519   
2017-08-14   5.407407  0.166667   6.240741  3237.629639  0.000000   
2017-08-15   6.240741  0.148148   6.277778  3162.462891  0.000000   

                                 

In [18]:
from sklearn.multioutput import RegressorChain

model = RegressorChain(base_estimator=XGBRegressor())
model.fit(X, y)

y_pred = pd.DataFrame(
    model.predict(X),
    index=y.index,
    columns=y.columns,
).clip(0.0)

In [19]:
print(y_pred.head())
prediction_date = '2017-08-15'
prediction_for_date = y_pred.loc[prediction_date]

nan_count = prediction_for_date.isna().sum()

print("Prediction for date:", prediction_date)
print(prediction_for_date.shape)
print("Number of NaN values:", nan_count)


                       y_step_1    y_step_2    y_step_3    y_step_4  \
date       family                                                     
2017-01-01 AUTOMOTIVE  2.581642  750.028992  565.400391  461.842926   
           BABY CARE   2.581642  750.028992  565.400391  461.842926   
           BEAUTY      2.581642  750.028992  565.400391  461.842926   
           BEVERAGES   2.581642  750.028992  565.400391  461.842926   
           BOOKS       2.581642  657.422913  570.554565  458.706055   

                         y_step_5   y_step_6    y_step_7    y_step_8  \
date       family                                                      
2017-01-01 AUTOMOTIVE  407.131470  436.78595  574.499695  548.103455   
           BABY CARE   407.131470  436.78595  574.499695  548.103455   
           BEAUTY      407.131470  436.78595  574.499695  548.103455   
           BEVERAGES   407.131470  436.78595  574.499695  548.103455   
           BOOKS       423.091888  436.78595  574.499695  520.701538  

In [20]:
FAMILY = 'BEAUTY'
START = '2017-08-15'
EVERY = 16

y_pred_ = y_pred.xs(FAMILY, level='family', axis=0).loc[START:]
y_ = family_sales.loc[START:, 'sales'].loc[:, FAMILY]




In [24]:
print(y)

                                          y_step_1     y_step_2     y_step_3  \
date       family                                                              
2017-01-01 AUTOMOTIVE                     0.092593    11.481482     8.296296   
           BABY CARE                      0.037037     0.259259     0.296296   
           BEAUTY                         0.055556    11.648149     7.185185   
           BEVERAGES                     74.222221  6208.055664  4507.814941   
           BOOKS                          0.000000     0.481481     0.814815   
...                                            ...          ...          ...   
2017-08-15 POULTRY                      325.679840     0.000000     0.000000   
           PREPARED FOODS                85.954132     0.000000     0.000000   
           PRODUCE                     2316.832764     0.000000     0.000000   
           SCHOOL AND OFFICE SUPPLIES    46.851852     0.000000     0.000000   
           SEAFOOD                      